In [2]:
import matplotlib as plt
import numpy as np
import pandas as pd

# from sklearn.pipeline import FeatureUnion, Pipeline
# from sklearn.base import TransformerMixin, BaseEstimator
# from sklearn.linear_model import LinearRegression
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.svm import SVR, LinearSVR
# from sklearn.decomposition import PCA, NMF
# from sklearn.feature_selection import SelectKBest, chi2
# from sklearn.preprocessing import StandardScaler, MinMaxScaler
# from sklearn.metrics import mean_squared_error
# from sklearn.model_selection import RepeatedKFold, cross_val_score, cross_val_predict, train_test_split, GridSearchCV, RandomizedSearchCV
# from scipy.stats import randint as sp_randint

In [5]:
from matminer.datasets import get_available_datasets, load_dataset, get_all_dataset_info

# load supercon databse and metadata
# print(get_all_dataset_info("superconductivity2018")) # metadata
data = pd.DataFrame(load_dataset('superconductivity2018'))
data.head()

#features will be made with matminer, target is Tc

,composition,Tc
0,Ba0.4K0.6Fe2As2,31.20
1,Ca0.4Ba1.25La1.25Cu3O6.98,40.10
2,Mo0.39Ru0.61,6.90
3,Tm4Os6Sn19,1.10
4,Nd1Bi0.99Pb0.01S2F0.3O0.7,4.85


In [9]:
from matminer.featurizers.structure import DensityFeatures
from matminer.featurizers.conversions import StrToComposition

# densityf = DensityFeatures()
# densityf.feature_labels()

# df = densityf.featurize_dataframe(df, "structure")
# df.head()

stc = StrToComposition()
df = stc.featurize_dataframe(data, "formula", pbar=False)

AttributeError: 'DataFrame' object has no attribute 'featurize_dataframe'